# This notebook explores the results for the agent based model

In [1]:
import re
import glob
import json
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option( 'display.max_columns', None )
pd.set_option( 'display.max_rows', None )

# Loading localidades GeoJSON for reference

# Loading ABM results

In [27]:
city = 'medellin'
scenario = 'no-cuarentena'
scenario_name = 'Sin cuarentena'
init_date = pd.to_datetime( '03/06/2020' )

In [28]:
path = './data/agents/{}/{}/*.csv'.format( city, scenario )

In [29]:
files = glob.glob( path )

In [30]:
files.sort()

In [31]:
agents = []
exp = 0

for file in files:
    df = pd.read_csv( file )
    df[ 'Fecha' ] = df[ 'Día' ].apply( lambda x: init_date + pd.DateOffset( days = x ) )
    df[ 'exp' ] = exp
    agents.append( df )
    exp = exp + 1

agents = pd.concat( agents )
del agents[ 'Unnamed: 0' ]

In [32]:
agents.shape

(9000, 44)

In [33]:
agents.dtypes

Susceptibles                                   int64
Total infectados                               int64
Graves                                         int64
Críticos                                       int64
Leves                                          int64
Recuperados                                    int64
Rt                                           float64
Recuento_zonas                                object
0-9                                           object
10-19                                         object
20-39                                         object
40-59                                         object
>60                                           object
En_cuarentena                                 object
Restringido_movilidad                         object
Cuarentena_permanente                         object
Vivos                                          int64
Día                                            int64
Contactos_prom_trabajo                       f

In [34]:
agents.tail()

,Susceptibles,Total infectados,Graves,Críticos,Leves,Recuperados,Rt,Recuento_zonas,0-9,10-19,20-39,40-59,>60,En_cuarentena,Restringido_movilidad,Cuarentena_permanente,Vivos,Día,Contactos_prom_trabajo,Contactos_prom_transporte,Contactos_prom_casa,Contactos_prom_trabajo_conm,Contactos_prom_transporte_conm,Contactos_prom_casa_conm,Nuevos_infectados,Detectados,En_testing,En_cama,En_UCI,Detectados_por_intervencion,#Intervenidos,porcentaje_edad_activa_por_zona,porcentaje_edad_activa,tasa_letalidad_por_zona,tasa_letalidad,total_muertos,total_muertos_por_zona,total_muertos_edad_activa,total_muertos_edad_activa_por_zona,zonas intervenidas,Graves_efectivos,Criticos_efectivos,Fecha,exp
295,26843,0,0,0,0,18255,0.0,"{13: [3077, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2299, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2441, ...","[3183, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3225, ...","[9549, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6153, ...","[7201, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4124, ...","[4611, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2312, ...","[0, 0, 0, 0, 0]","[4568, 0, 0, 0, 3049]","[0, 0, 0, 0, 0]",45098,295,0.553018,1.787019,0.926804,0.665404,2.150183,0.929644,0,428,0,56,9,0,0,"{1: 0.01272264631043257, 2: 0.0118289353958143...",0.005515,"{1: 0.07142857142857142, 2: 0.0930232558139534...",0.10084,48,"{1: 4, 2: 4, 3: 9, 4: 5, 5: 4, 6: 11, 7: 3, 8:...",6,"{1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 2, 7: 1, 8: ...","{5: [[116, 15], [117, 15], [118, 15], [119, 15...",0,0,2020-12-26,29
296,26843,0,0,0,0,18255,0.0,"{13: [3077, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2299, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2441, ...","[3183, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3225, ...","[9549, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6153, ...","[7201, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4124, ...","[4611, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2312, ...","[0, 0, 0, 0, 0]","[7142, 0, 0, 0, 4826]","[0, 0, 0, 0, 0]",45098,296,0.430396,1.417025,0.915406,0.585874,1.928916,0.915816,0,428,0,56,9,0,0,"{1: 0.01272264631043257, 2: 0.0118289353958143...",0.005515,"{1: 0.07142857142857142, 2: 0.0930232558139534...",0.10084,48,"{1: 4, 2: 4, 3: 9, 4: 5, 5: 4, 6: 11, 7: 3, 8:...",6,"{1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 2, 7: 1, 8: ...","{5: [[116, 15], [117, 15], [118, 15], [119, 15...",0,0,2020-12-27,29
297,26843,0,0,0,0,18255,0.0,"{13: [3077, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2299, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2441, ...","[3183, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3225, ...","[9549, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6153, ...","[7201, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4124, ...","[4611, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2312, ...","[0, 0, 0, 0, 0]","[4704, 0, 0, 0, 3334]","[0, 0, 0, 0, 0]",45098,297,0.536720,1.760100,0.923123,0.653130,2.141851,0.925256,0,428,0,56,9,0,0,"{1: 0.01272264631043257, 2: 0.0118289353958143...",0.005515,"{1: 0.07142857142857142, 2: 0.0930232558139534...",0.10084,48,"{1: 4, 2: 4, 3: 9, 4: 5, 5: 4, 6: 11, 7: 3, 8:...",6,"{1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 2, 7: 1, 8: ...","{5: [[116, 15], [117, 15], [118, 15], [119, 15...",0,0,2020-12-28,29
298,26843,0,0,0,0,18255,0.0,"{13: [3077, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2299, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2441, ...","[3183, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3225, ...","[9549, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6153, ...","[7201, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4124, ...","[4611, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2312, ...","[0, 0, 0, 0, 0]","[5067, 0, 0, 0, 3391]","[0, 0, 0, 0, 0]",45098,298,0.525234,1.711828,0.918400,0.646479,2.106987,0.919241,0,428,0,56,9,0,0,"{1: 0.01272264631043257, 2: 0.0118289353958143...",0.005515,"{1: 0.07142857142857142, 2: 0.0930232558139534...",0.10084,48,"{1: 4, 2: 4, 3: 9, 4: 5, 5: 4, 6: 11, 7: 3, 8:...",6,"{1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 2, 7: 1, 8: ...","{5: [[116, 15], [117, 15], [118, 15], [119, 15...",0,0,2020-12-29,29
299,26843,0,0,0,0,18255,0.0,"{13: [3077, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2299, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2441, ...","[3183, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3225, ...","[9549, 

In [35]:
def sum_quarantine( x ):
    x = json.loads( x )
    people_in_q = x[ 0 ] + x[ 1 ] + x[ 2 ] + x[ 3 ] + x[ 4 ]
    return people_in_q

In [36]:
agents_norm = []

for exp in agents[ 'exp' ].unique():
    df = agents.loc[ agents[ 'exp' ] == exp ]
    total_people = df[ 'Vivos' ].values[ 0 ]
    df[ 'Graves' ] = df[ 'Graves' ] / total_people
    df[ 'Críticos' ] = df[ 'Críticos' ] / total_people
    df[ 'Fallecidos' ] = ( total_people - df[ 'Vivos' ] ) / total_people
    df[ 'R0' ] = ( df[ 'Nuevos_infectados' ].cumsum() / total_people )
    df[ 'Cuarentena' ] = df[ 'En_cuarentena' ].apply( lambda x: sum_quarantine( x ) ) / total_people
    agents_norm.append( df[ [ 'exp', 'Fecha', 'Graves', 'Críticos', 'Fallecidos', 'R0', 'Rt', 'Cuarentena' ] ] )

agents_norm = pd.concat( agents_norm )

/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [37]:
agents_norm.tail()

,exp,Fecha,Graves,Críticos,Fallecidos,R0,Rt,Cuarentena
295,29,2020-12-26,0.0,0.0,0.001063,0.405285,0.0,0.0
296,29,2020-12-27,0.0,0.0,0.001063,0.405285,0.0,0.0
297,29,2020-12-28,0.0,0.0,0.001063,0.405285,0.0,0.0
298,29,2020-12-29,0.0,0.0,0.001063,0.405285,0.0,0.0
299,29,2020-12-30,0.0,0.0,0.001063,0.405285,0.0,0.0


In [38]:
var = 'Críticos'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'ucis' ), index = False )

var = 'Fallecidos'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'deaths' ), index = False )

var = 'R0'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'r0' ), index = False )

var = 'Rt'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'rt' ), index = False )

var = 'Cuarentena'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'quarantine' ), index = False )

In [ ]:
def ci( x ):
    try:
        return 1.96 * np.std( x ) / np.sqrt( len( x ) )
    except ZeroDivisionError:
        return 0

agents_norm_grouped = agents_norm.groupby( 'Fecha' ).agg( { 'Graves': [ 'mean', ci ], 'Críticos': [ 'mean', ci ], 'Fallecidos': [ 'mean', ci ], 'R0': [ 'mean', ci ], 'Rt': [ 'mean', ci ], 'Cuarentena': [ 'mean' ] } )
agents_norm_grouped.columns = [ '_'.join( col ).strip() for col in agents_norm_grouped.columns.values ]

In [ ]:
agents_norm_grouped.tail()

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Graves_mean' ], label = 'Graves', color = '#317372' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Graves_mean' ] - agents_norm_grouped[ 'Graves_ci' ] ), ( agents_norm_grouped[ 'Graves_mean' ] + agents_norm_grouped[ 'Graves_ci' ] ), color = '#317372', alpha = .1 )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Críticos_mean' ], label = 'Críticos', color = '#f98a4b' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Críticos_mean' ] - agents_norm_grouped[ 'Críticos_ci' ] ), ( agents_norm_grouped[ 'Críticos_mean' ] + agents_norm_grouped[ 'Críticos_ci' ] ), color = '#f98a4b', alpha = .1 )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Fallecidos_mean' ], label = 'Fallecidos', color = '#f64438' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Fallecidos_mean' ] - agents_norm_grouped[ 'Fallecidos_ci' ] ), ( agents_norm_grouped[ 'Fallecidos_mean' ] + agents_norm_grouped[ 'Fallecidos_ci' ] ), color = '#f64438', alpha = .1 )

plt.title( 'Graves / Críticos / Fallecidos' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'Casos' )
plt.grid( True )
plt.legend()

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'R0_mean' ] )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'R0_mean' ] - agents_norm_grouped[ 'R0_ci' ] ), ( agents_norm_grouped[ 'R0_mean' ] + agents_norm_grouped[ 'R0_ci' ] ), alpha = .1 )

plt.title( 'Tasa de ataque' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'R0' )
plt.grid( True )

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Rt_mean' ] )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Rt_mean' ] - agents_norm_grouped[ 'Rt_ci' ] ), ( agents_norm_grouped[ 'Rt_mean' ] + agents_norm_grouped[ 'Rt_ci' ] ), alpha = .1 )

plt.title( 'Tasa de reproducción afectiva' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'Rt' )
plt.grid( True )

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Cuarentena_mean' ] )

plt.title( 'Cuarentena' )
plt.xlabel( 'Fecha' )
plt.ylabel( '% cuarentena' )
plt.grid( True )

In [ ]:
agents_norm.tail()

In [ ]:
agents_norm[ [ 'exp', 'Fecha', 'Graves', 'Críticos', 'Fallecidos', 'R0', 'Rt', 'Cuarentena' ] ].to_csv( './data/agents/clean/{}-{}.csv'.format( city, scenario ), index = False )